# Generating LLM Perspectives & Comparing to Human Perspectives
So far, we've been using the human provided responses as the "ground truth" for what is the full overton window. However, we can also generate our own "ground truth" by asking LLMs to provide their own perspectives.

We're then going to compare the mapping of the LLM perspectives to the human provided responses.

This takes in the `habermas_machine_questions_with_responses.csv` file and generates responses from the LLMs and will update it with the new LLM perspectives.

In [ ]:
from dotenv import load_dotenv
import pandas as pd, numpy as np, os

# Load environment variables
load_dotenv()
DATA_PATH = os.getenv('DATA_PATH')
TEMP_PATH = os.getenv('TEMP_PATH')

df_questions = pd.read_csv(DATA_PATH+'habermas_machine_questions_with_responses.csv')

In [ ]:
df_questions.head()

## Structured Generation of LLM Perspectives

In [ ]:

from pydantic import BaseModel
from openai import OpenAI
import json

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

class SinglePerspective(BaseModel):
    perspective: str

class PerspectiveChain(BaseModel):
    steps: list[SinglePerspective]

def generate_perspectives(question: str):
    """
    Generate a multiple perspectives of answers to a question.

    Args:
        question: Context question or query
    """
    system_prompt = f"""
You are a helpful assistant that generates multiple perspectives of answers to a question. You will be given a question and you will generate a list of possible answer perspectives. Make sure you cover all possible perspectives but do not repeat yourself.
"""

    prompt = f"""Question: {question}
Now, step by step, outline each broad answer perspective to this question."""

    chat_response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        response_format={
            'type': 'json_schema',
            'json_schema': 
                {
                "name": "PerspectiveChain", 
                "schema": PerspectiveChain.model_json_schema()
                }
            } 
    )

    result_object = json.loads(chat_response.choices[0].message.content)
    return [step['perspective'] for step in result_object['steps']]


In [ ]:
from tqdm import tqdm

# Using a loop rather than .apply for restart simplicity & tqdm
llm_perspectives = []
for question in tqdm(df_questions['question.text'], desc="Generating perspectives"):
    perspectives = generate_perspectives(question)
    llm_perspectives.append(perspectives)

df_questions['llm_perspectives'] = llm_perspectives

In [ ]:
df_llm_perspectives = df_questions[['question.text', 'question_topic', 'question_id', 'llm_perspectives']]
df_llm_perspectives.head()

In [ ]:
# To keep things separate and clean, we're going to save these to a different file.
df_llm_perspectives.to_csv(DATA_PATH+'habermas_machine_questions_with_LLM_generated_perspectives.csv', index=False) 

# Comparative Analysis



In [ ]:
import ast

df_llm_perspectives = pd.read_csv(DATA_PATH+'habermas_machine_questions_with_LLM_generated_perspectives.csv')
df_questions = pd.read_csv(DATA_PATH+'habermas_machine_questions_with_responses.csv')
df_questions['own_opinion.text'] = df_questions['own_opinion.text'].apply(ast.literal_eval)
df_llm_perspectives['llm_perspectives'] = df_llm_perspectives['llm_perspectives'].apply(ast.literal_eval)

In [ ]:
# Match the df_llm_perspectives and df_questions by question_id
df_llm_perspectives = df_llm_perspectives.merge(df_questions[['question_id', 'own_opinion.text']], on='question_id', how='left')
df_llm_perspectives.head()

In [ ]:
from openai import OpenAI
import json

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def are_these_perspectives_the_same(perspective_a: str, perspective_b: str):
    """
    Determine if two perspectives are the same.
    """
    system_prompt = f"""You will be given two perspectives and you will determine if they are the same. Read carefully the two perspectives and answer yes if they are expressing the same broad perspective or opinion. Answer no otherwise. ONLY say a single word: 'yes' or 'no'."""

    prompt = f"""Perspective A: {perspective_a}
Perspective B: {perspective_b}
Are these the same perspective/opinion? Yes/no answer:"""

    chat_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=1
    )
    return 1 if chat_response.choices[0].message.content.strip().lower() == 'yes' else 0

In [ ]:
same_perspective_matrix = [[0]*len(row['own_opinion.text'])]*len(row['llm_perspectives'])

In [ ]:
same_perspective_matrix

In [ ]:
for index, row in df_llm_perspectives.iterrows():
    print(row['question.text'])
    print(row['own_opinion.text'])
    # We're gonna generate a pairwise comparison of the own opinion and each of the LLM perspectives.
    # The complexity of this is pretty bad given the size of the context windows and can be very slow (20s per question).
    same_perspective_matrix = [[0]*len(row['own_opinion.text'])]*len(row['llm_perspectives'])
    for i, perspective in enumerate(row['llm_perspectives']):
        for j, own_opinion in enumerate(row['own_opinion.text']):
            same_perspective_matrix[i][j] = are_these_perspectives_the_same(perspective, own_opinion)
    print(same_perspective_matrix)
    break

In [ ]:
row['llm_perspectives'][2]

In [ ]:
row['own_opinion.text'][4]

### WIP Notes

It's really hard to actually determine if two perspectives are the same. We're going to need to try to interate on this.

In addition, using the LLM to do so is reasonable slow (but not too bad). Perhaps a clustering method would be better?